In [2]:
# S-Box (fixed mapping based on the table)
S_BOX = {
    0x0: 0xE, 0x1: 0x4, 0x2: 0xD, 0x3: 0x1,
    0x4: 0x2, 0x5: 0xF, 0x6: 0xB, 0x7: 0x8,
    0x8: 0x3, 0x9: 0xA, 0xA: 0x6, 0xB: 0xC,
    0xC: 0x5, 0xD: 0x9, 0xE: 0x0, 0xF: 0x7
}

#P-box permutation (transposition of bits)
P_BOX = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

# Function To apply S-box substitution(4 bit input to 4 bit output)
def s_box_substitution(input_bits):
    return S_BOX[input_bits]

# Function to apply the P-box permutation (input is a 16 bit block)
def p_box_permutation(input_bits):
    permuted_bits = 0
    for i, p in enumerate(P_BOX):
        bit = (input_bits >> (16 - p)) & 1 # Extract the bits at position p
        permuted_bits |= bit << (15 - i) # Set the bit at the new position
    return permuted_bits

# Funtion to apply XOR operation with subkey
def xor_with_subkey(input_bits, subkey):
    return input_bits ^ subkey

def round_function(data, subkey):
    # Break the 16 bit data into 4 bit sub blocks
    sub_blocks = [(data >> (12 - 4 * i)) & 0xF for i in range(4)]
    
    # Apply S-box substitution to each sub block
    sub_blocks = [s_box_substitution(block) for block in sub_blocks]
    
    # Recombine the sub blocks into a 16 bit block
    substitued_data = 0
    for i in range(4):
        substitued_data |= (sub_blocks[i] << (12 - 4 * i))
    
    # Apply P-box permutation
    permuted_data = p_box_permutation(substitued_data)
    
    # Apply XOR with subkey
    return xor_with_subkey(permuted_data, subkey)

def encrypt(data, subkeys):
    # Ensure the input data is 16 bit integer
    assert 0 <= data < 2**16, "Input data must be a 16-bit integer"
    
    # Apply the round function for each round
    for i in range(3):
        data = round_function(data, subkeys[i])
        
    # Return the final ciphertext
    return data

def main():
    # Example usage
    plaintext = 0x1234
    
    # Subkeys for each round(e.g., 16-bit subkeys)
    subkeys = [0x1F2A, 0x3C4D, 0x5E6F]
    
    #Encrypt the plaintext
    ciphertext = encrypt(plaintext, subkeys)
    
    print("Plaintext: 0x{:04X}".format(plaintext))
    print("Ciphertext: 0x{:04X}".format(ciphertext))
    
if __name__ == "__main__":
    main()

Plaintext: 0x1234
Ciphertext: 0x8128


In [3]:
# Inverse S-box mapping
INVERSE_S_BOX = {v: k for k, v in S_BOX.items()}

# Function to apply inverse S-box substitution
def inverse_s_box_substitution(input_bits):
    return INVERSE_S_BOX[input_bits]

# Function to apply inverse P-box permutation
def inverse_p_box_permutation(input_bits):
    permuted_bits = 0
    for i in range(16):
        # Find the original position of the current bit
        original_pos = P_BOX.index(i + 1)
        # Extract the bit from its current position
        bit = (input_bits >> (15 - i)) & 1
        # Place it in its original position
        permuted_bits |= bit << (15 - original_pos)
    return permuted_bits

# XOR operation is its own inverse
def xor_with_subkey(input_bits, subkey):
    return input_bits ^ subkey

def inverse_round_function(data, subkey):
    # First undo the XOR with subkey
    data = xor_with_subkey(data, subkey)
    
    # Then undo the P-box permutation
    data = inverse_p_box_permutation(data)
    
    # Break the 16 bit data into 4 bit sub blocks
    sub_blocks = [(data >> (12 - 4 * i)) & 0xF for i in range(4)]
    
    # Apply inverse S-box substitution to each sub block
    sub_blocks = [inverse_s_box_substitution(block) for block in sub_blocks]
    
    # Recombine the sub blocks into a 16 bit block
    decrypted_data = 0
    for i in range(4):
        decrypted_data |= (sub_blocks[i] << (12 - 4 * i))
    
    return decrypted_data

def decrypt(ciphertext, subkeys):
    # Ensure the input ciphertext is 16 bit integer
    assert 0 <= ciphertext < 2**16, "Input ciphertext must be a 16-bit integer"
    
    # Process subkeys in reverse order
    for i in reversed(range(3)):
        ciphertext = inverse_round_function(ciphertext, subkeys[i])
    
    return ciphertext

def main():
    # Example usage
    plaintext = 0x1234
    # Subkeys for each round (e.g., 16-bit subkeys)
    subkeys = [0x1F2A, 0x3C4D, 0x5E6F]
    
    # Encrypt the plaintext
    ciphertext = encrypt(plaintext, subkeys)
    # Decrypt the ciphertext
    decrypted = decrypt(ciphertext, subkeys)
    
    print("Plaintext:      0x{:04X}".format(plaintext))
    print("Ciphertext:     0x{:04X}".format(ciphertext))
    print("Decrypted text: 0x{:04X}".format(decrypted))

if __name__ == "__main__":
    main()

Plaintext:      0x1234
Ciphertext:     0x8128
Decrypted text: 0x1234
